In [1]:
import os

In [2]:
from langchain_groq import ChatGroq
from langchain.schema import HumanMessage, SystemMessage

llm_model = ChatGroq(
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model_name="llama3-70b-8192"
)


In [4]:
from langchain_core.prompts import PromptTemplate

from langchain.output_parsers.json import SimpleJsonOutputParser

In [5]:
json_prompt = PromptTemplate.from_template("Return a json object with an 'answer' key that answers the following question : {question}")

In [6]:
json_parser = SimpleJsonOutputParser()

In [7]:
json_chain = json_prompt | llm_model | json_parser

In [9]:
json_parser.get_format_instructions()

'Return a JSON object.'

In [10]:
json_chain.invoke({"question": "What is the biggest country?"})

{'answer': 'Russia'}

## Custom Output Parsers by usnig Pydantic

In [11]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

In [12]:
class Joke(BaseModel):
    Setup: str = Field(description="question to set up a joke")
    punchline : str = Field(description="answer to resolve the joke")

In [14]:
parser = JsonOutputParser(pydantic_object=Joke)

In [23]:
prompt = PromptTemplate(
    template = "Answer the user query. \n {format_instructions} \n {query}",
    input_variables=["query"],
    partial_variables  = {"format_instructions" : parser.get_format_instructions()},
)


In [24]:
chain = prompt | llm_model | parser

In [25]:
chain.invoke({"query": "tell me a joke"})

{'Setup': 'Why was the math book sad?',
 'punchline': 'Because it had too many problems.'}